In [14]:
from torchvision.models import resnet18
from torchvision import transforms
import torchvision
import time
import torch
import torch.nn as nn

model_ft = resnet18(num_classes=10)
model_ft.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False)

trainingBatchSize = 128

transform_train = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

transform_test = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

trainset = torchvision.datasets.SVHN(
    root="./data", split="train", download=True, transform=transform_train
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=trainingBatchSize, shuffle=True, num_workers=2
)

testset = torchvision.datasets.SVHN(
    root="./data", split="test", download=True, transform=transform_test
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2
)  ##test batch size is kept fized at 100 and not varied


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


criterion = nn.CrossEntropyLoss()


def train(train_loader, model, criterion, epoch, l1=False):

    losses = AverageMeter()
    top1 = AverageMeter()

    model.cuda()
    # switch to train mode
    model.train()

    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), 0.01)

    for j in range(epoch):
        for i, (image, target) in enumerate(train_loader):

            image = image.cuda()
            target = target.cuda()

            # compute output
            output_clean = model(image)

            loss = criterion(output_clean, target)
            # if l1:
            #     loss = loss + args.alpha * l1_regularization(model)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            output = output_clean.float()
            loss = loss.float()
            # measure accuracy and record loss
            prec1 = accuracy(output.data, target)[0]

            losses.update(loss.item(), image.size(0))
            top1.update(prec1.item(), image.size(0))

            if (i + 1) % 100 == 0:
                end = time.time()
                print(
                    "Epoch: [{0}][{1}/{2}]\t"
                    "Loss {loss.val:.4f} ({loss.avg:.4f})\t"
                    "Accuracy {top1.val:.3f} ({top1.avg:.3f})\t"
                    "Time {3:.2f}".format(
                        j, i, len(train_loader), end - start, loss=losses, top1=top1
                    )
                )
                start = time.time()

        print("train_accuracy {top1.avg:.3f}".format(top1=top1))

    return top1.avg

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


In [15]:
train(trainloader, model_ft, criterion, 100)

Epoch: [0][99/573]	Loss 2.2203 (2.2663)	Accuracy 21.875 (17.453)	Time 1.84
Epoch: [0][199/573]	Loss 2.2673 (2.2575)	Accuracy 20.312 (17.914)	Time 1.60
Epoch: [0][299/573]	Loss 2.2781 (2.2469)	Accuracy 15.625 (18.565)	Time 1.62
Epoch: [0][399/573]	Loss 2.1041 (2.2283)	Accuracy 26.562 (19.672)	Time 1.62
Epoch: [0][499/573]	Loss 1.9619 (2.1928)	Accuracy 30.469 (21.238)	Time 1.86
train_accuracy 22.585
Epoch: [1][99/573]	Loss 1.7700 (2.1049)	Accuracy 36.719 (24.821)	Time 3.11
Epoch: [1][199/573]	Loss 1.7137 (2.0468)	Accuracy 42.188 (27.113)	Time 1.54
Epoch: [1][299/573]	Loss 1.4109 (1.9888)	Accuracy 50.781 (29.281)	Time 1.53
Epoch: [1][399/573]	Loss 1.4635 (1.9353)	Accuracy 50.781 (31.276)	Time 1.72
Epoch: [1][499/573]	Loss 1.3859 (1.8873)	Accuracy 52.344 (33.075)	Time 1.91
train_accuracy 34.363
Epoch: [2][99/573]	Loss 1.2328 (1.8075)	Accuracy 53.906 (36.091)	Time 3.34
Epoch: [2][199/573]	Loss 1.1592 (1.7638)	Accuracy 55.469 (37.709)	Time 1.60
Epoch: [2][299/573]	Loss 1.2728 (1.7232)	Accura

KeyboardInterrupt: 

In [16]:
torch.save(model_ft.state_dict(), "svhn_ckpt.pth")